In [ ]:
import tensorflow as tf
import tensorflow as tf

# Transfer Learning
Transfer Learning a technique which is used to get the advantage of other pre_trained models in order to save time and help us in computation process.
the 2nd advantage is the accuracy, as we are not having access to wide variety of data to learn the parameters from, we might face overfitting in our model, by taking the advantage of transfer learning,which had trained on 1000s of images, we can increase the validation and training accuracy of our model.

# Inception V.3
It is an images recognition model which has beeen shown to attain greater than 78.1 % accuracy on imageNet dataset. this model itself builds upon pooling layers,Dense layers, activation funcitons,fully connected layers and so on.
 This model can classify images into 1000 categories,

In [ ]:
# Download the pre-trained weights. No top means it excludes the fully connected layer it uses for classification.
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
    

# Freezing and importing the inception model.
the main idea behind the freezing is to avoid retraining the pretrained weightts which we have already downloaded.
as we are going to add our dense layer to the end of some layer of the pretrained inception network layers, we will remove the last fully connected layer,because we will add our own layer at the end.

from model.summary(), we can observe the operations and parameters, we also can select till the which part of the network we want to use,for exampel here will use up to mixed_8 layer.

## Note:
Remember that the as much as we go deeper inside the network, that much our layers are going to get specific and specialized about the features

In [ ]:
# import the models
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers

# set the weights file which we have downloaded in previous layer into a variable
weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

# define the base model,set the input shape and remove the dense layer
pre_trained_model = InceptionV3(input_shape=(150,150,3),
                                include_top = False,
                                weights = None) 
# load the pre_trained weights in to the model
pre_trained_model.load_weights(weights_file)

# Freeze the weights
for layer in pre_trained_model.layers:
  layer.trainable = False

# pre_trained_model.summary()

Choose the last layer, as mentioned before, we are going with mixed_8 layer

In [ ]:
last_pre_trained_layer = pre_trained_model.get_layer('mixed8')
print('last layer output shape: ', last_pre_trained_layer.output_shape)
last_output = last_pre_trained_layer.output

# Add Dense layer to the network,
 Here by Dense layer we mean the layer we want to add from our side.


In [ ]:
from tensorflow.keras import Model

# Flatten the output layer to 1 dimension( the mixed8 one )
last_layer = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
last_layer = layers.Dense(1280,activation="relu")(last_layer)
# Add a dropout rate of 0.3
last_layer = layers.Dropout(0.3)(last_layer)
# Add a sigmoid function to classify the image after the Dense layer
last_layer = layers.Dense(1,activation="sigmoid")(last_layer)
# Append the Dense network to the base(pre_trained_model) model, this Dense network have been introduced by us.!
model = Model(pre_trained_model.input,last_layer)
# model.summary()


Set the parameters like, loss,optimizer and metrics for training

In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer = tf.keras.optimizers.Adam(),
              metrics = ['accuracy'])

# Prepare the data set
we are going to use the Imageprocessing technique to process our data and flow it frow our specific directories,and alse on the fly we will rescale and augment all the iamges before feeding into the network, the purpose of this is to avoid overfitting and introduce new variety of pictures.

In [ ]:
# Download the dataset
!wget https://storage.googleapis.com/tensorflow-1-public/course2/cats_and_dogs_filtered.zip

In [ ]:
from tensorflow.python.ops.gen_batch_ops import batch
import os 
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Extract the archive
path = "./cats_and_dogs_filtered.zip"
zip_ref = zipfile.ZipFile(path,'r')
zip_ref.extractall('tmp/')
zip_ref.close()

# Defining our base directory

base_dir = 'tmp/cats_and_dogs_filtered'

train_dir = os.path.join(base_dir,"train")
validation_dir = os.path.join(base_dir,'validation')

# directory with training cat pictures
train_cats_dir = os.path.join(train_dir,'cats')
# direcctory with training dogs
train_dogs_dir = os.path.join(train_dir,'dogs')
# directory with validation cat pictures
val_cats_dir = os.path.join(validation_dir,'cats')
# directory with validation dog pictures
val_dogs_dir = os.path.join(validation_dir,"dogs")

# Using ImageDataGenerator to augment and rescale our image on the fly(while we are flowing data,IDG will do the processing simultaneously)
Train_DataGenerator = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
# Validation data should not be augmented as we are checking the goodness of our model while training with this.
Val_DataGenerator = ImageDataGenerator(rescale = 1./255.)
# Flowing the training data from directory, targrt_size means that we might have different sizes of images,so we want all in one size.
Training_Data = Train_DataGenerator.flow_from_directory(train_dir,
                                                        batch_size = 20,
                                                        class_mode = 'binary',
                                                        target_size=(150,150))
# Flowing validation data
Validation_Data = Val_DataGenerator.flow_from_directory(validation_dir,
                                                        batch_size = 20,
                                                        class_mode = 'binary',
                                                        target_size = (150,150))



# Train the model


In [ ]:
history = model.fit(Training_Data,
                    epochs = 10,
                    validation_data = Validation_Data,
                    verbose  = 2)

# Note
the fit() method returns a History object containing the traning parameters(histoty.params),the list of epohcs it went through(history.epoch), and importantly a dictionary (history.history) containing the loss and extra metrics it measured at the end of each epoch on the training set and validation set.

# Plot the accuracy and loss of our model

In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(accuracy))

plt.plot(epochs,accuracy,'r',label = 'Training_accuracy')
plt.plot(epochs,val_accuracy,'g',label = 'Validation_accuracy')
plt.title('Validation and Traininig accuracy')
plt.legend()
plt.figure()

# Plotting the loss for both training and validation data
plt.plot(epochs,loss,'r',label = 'Training_loss')
plt.plot(epochs,val_loss,'g',label = 'Validation_loss')
plt.legend(loc=0)
plt.figure()

plt.show()


In [ ]:
import pandas as pd
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()


In [ ]:
|